In [68]:
import numpy as np
import pandas as pd

import tensorflow as tf
import matplotlib.pyplot as plt
import keras 

from keras.layers import Input, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split

"""
    Created by Mohsen Naghipourfar on 3/26/18.
    Email : mn7697np@gmail.com
    Website: http://ce.sharif.edu/~naghipourfar
"""

'\n    Created by Mohsen Naghipourfar on 3/26/18.\n    Email : mn7697np@gmail.com\n    Website: http://ce.sharif.edu/~naghipourfar\n'

In [83]:
# Hyper-Parameters
LEARNING_RATE = 1e-3
DROP_OUT = 0.5
N_SAMPLES = 10787
N_FEATURES = 19671
N_DISEASES = 34
N_BATCHES = 2000
N_EPOCHS = 50
N_BATCH_LEARN = 10
N_RANDOM_FEATURES = 200
neurons = {
    'in': 200,
    'l1': 1024,
    'l2': 512,
    'l3': 256,
    'l4': 128,
    'out': N_DISEASES
}


In [75]:
def modify_output(target):
    global N_DISEASES
    output_dict = {}
    a = target[0].value_counts()
    # print(a)
    a = pd.DataFrame(a)
    i = 0
    for row in a.itertuples():
        output_dict[row[0]] = i
        i += 1
    N_DISEASES = i
    new_output = [0 for __ in range(N_SAMPLES)]
#     new_output = [[0 for _ in range(N_DISEASES)] for __ in range(N_SAMPLES)]
    for idx, y in target.iterrows():
#         new_output[idx][output_dict[y[0]]] = 1
        new_output[idx] = output_dict[y[0]]
    return new_output

In [78]:
# Load Data
x_data = pd.read_csv("../Data/fpkm_normalized.csv", header=None)
# y_data = pd.read_csv("../Data/disease.csv", header=None)
# y_data = pd.DataFrame(modify_output(y_data))

In [79]:
y_data = pd.read_csv("../Data/disease.csv", header=None)
y_data = pd.DataFrame(modify_output(y_data))
y_data = pd.DataFrame(keras.utils.to_categorical(y_data, num_classes=N_DISEASES))

In [80]:
# Random Feature Selection
random_feature_indices = np.random.choice(19671, N_RANDOM_FEATURES, replace=False)
x_data_new = x_data[random_feature_indices]

# Train/Test Split
x_train, x_test, y_train, y_test = train_test_split(x_data_new, y_data, test_size=0.20)

In [44]:
# x_train.columns = range(x_train.shape[1])
# x_test.columns = range(x_test.shape[1])

False

In [81]:
# Design Model
input_layer = Input(shape=(neurons['in'],))

l1 = Dense(neurons['l1'], activation='relu')(input_layer)

l2 = Dense(neurons['l2'], activation='relu')(l1)

l3 = Dense(neurons['l3'], activation='relu')(l2)

l4 = Dense(neurons['l4'], activation='relu')(l3)

output_layer = Dense(neurons['out'], activation='softmax')(l4)

# Compile Model
network = Model(input_layer, output_layer)

network.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

network.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 200)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 1024)              205824    
_________________________________________________________________
dense_37 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_38 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_39 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_40 (Dense)             (None, 34)                4386      
Total params: 899,234
Trainable params: 899,234
Non-trainable params: 0
_________________________________________________________________


/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspe

In [51]:
from keras.callbacks import Callback
class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [87]:
# # Train Model
network.fit(x=x_train.as_matrix(),
            y=y_train.as_matrix(),
            epochs=N_EPOCHS,
            batch_size=N_BATCHES,
            shuffle=True,
            validation_data=(x_test.as_matrix(), y_test.as_matrix())
            )

# TODO: Use Callback function for our network!

Train on 8629 samples, validate on 2158 samples
Epoch 1/50
8629/8629 [==============================] - 0s - loss: 0.0240 - acc: 0.9955 - val_loss: 0.5478 - val_acc: 0.8906
Epoch 2/50
8629/8629 [==============================] - 0s - loss: 0.0241 - acc: 0.9956 - val_loss: 0.5407 - val_acc: 0.8962
Epoch 3/50
8629/8629 [==============================] - 0s - loss: 0.0255 - acc: 0.9946 - val_loss: 0.5421 - val_acc: 0.8930
Epoch 4/50
8629/8629 [==============================] - 0s - loss: 0.0249 - acc: 0.9939 - val_loss: 0.5443 - val_acc: 0.8934
Epoch 5/50
8629/8629 [==============================] - 0s - loss: 0.0252 - acc: 0.9934 - val_loss: 0.5410 - val_acc: 0.8939
Epoch 6/50
8629/8629 [==============================] - 0s - loss: 0.0229 - acc: 0.9956 - val_loss: 0.5455 - val_acc: 0.8906
Epoch 7/50
8629/8629 [==============================] - 0s - loss: 0.0325 - acc: 0.9898 - val_loss: 0.5821 - val_acc: 0.8804
Epoch 8/50
8629/8629 [==============================] - 0s - loss: 0.0336 - a

In [112]:
# keras.utils.plot_model(network, to_file='model.png', show_shapes=False, show_layer_names=True, rankdir='TB')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

b"%!PS-Adobe-3.0\n%%Creator: graphviz version 2.40.1 (20161225.0304)\n%%Title: G\n%%Pages: (atend)\n%%BoundingBox: (atend)\n%%EndComments\nsave\n%%BeginProlog\n/DotDict 200 dict def\nDotDict begin\n\n/setupLatin1 {\nmark\n/EncodingVector 256 array def\n EncodingVector 0\n\nISOLatin1Encoding 0 255 getinterval putinterval\nEncodingVector 45 /hyphen put\n\n% Set up ISO Latin 1 character encoding\n/starnetISO {\n        dup dup findfont dup length dict begin\n        { 1 index /FID ne { def }{ pop pop } ifelse\n        } forall\n        /Encoding EncodingVector def\n        currentdict end definefont\n} def\n/Times-Roman starnetISO def\n/Times-Italic starnetISO def\n/Times-Bold starnetISO def\n/Times-BoldItalic starnetISO def\n/Helvetica starnetISO def\n/Helvetica-Oblique starnetISO def\n/Helvetica-Bold starnetISO def\n/Helvetica-BoldOblique starnetISO def\n/Courier starnetISO def\n/Courier-Oblique starnetISO def\n/Courier-Bold starnetISO def\n/Courier-BoldOblique starnetISO def\ncleartoma